# Cleaned Data set merge code

This file merges previously cleaned data sets, and creates the master file for analysis.

## Imports

In [55]:
import pandas as pd
import seaborn as sn
import pandas_profiling as pp
import numpy as np
from matplotlib import pyplot as pl
%matplotlib tk

In [10]:
product1 = pd.read_csv("cleaned_products.csv")
store1 = pd.read_csv("cleaned_store.csv")
transact1 = pd.read_csv("cleaned_transaction.csv")

### Index was promoted as a column into the CSV files and needs to be dropped

In [18]:
transact1.head(1)

,DAY_DT,LOC_IDNT,DBSKU,FULL_PRICE_IND,TOTAL_SALES,TOTAL_UNITS,TOTAL_SALES_PRFT,TOTAL_COST,UNIT_SALES_PRICE
0,2015-09-26,1218,466896.0,0,16.8,1.0,1.3,15.5,16.8


In [16]:
store1.head(1)

,Unnamed: 0,LOC_IDNT,CITY,STATE,STORE_TYPE,POSTAL_CD,STORE_SIZE
0,0,249,ST LOUIS,MO,Strip Store,63119,3963.0


In [17]:
product1.head(1)

,Unnamed: 0,DBSKU,DEPARTMENT,CLASS,SUBCLASS,SUBCLASS_NAME,DSUBCLASS
0,0,2182204.0,12,3,32,1,1232


#### Drop Headers

In [ ]:
transact1=transact1.drop('Unnamed: 0',axis=1)

In [19]:
store1=store1.drop('Unnamed: 0',axis=1)

In [20]:
product1=product1.drop('Unnamed: 0',axis=1)

## Merge Transaction and Store data

In [33]:
transact2=transact1.merge(store1,how='left',on="LOC_IDNT")

In [34]:
round(100*(transact2.isnull().sum()/len(transact2.index)), 2)

DAY_DT              0.00
LOC_IDNT            0.00
DBSKU               0.00
FULL_PRICE_IND      0.00
TOTAL_SALES         0.00
TOTAL_UNITS         0.00
TOTAL_SALES_PRFT    0.00
TOTAL_COST          0.00
UNIT_SALES_PRICE    0.00
CITY                0.92
STATE               0.92
STORE_TYPE          0.92
POSTAL_CD           0.92
STORE_SIZE          0.92
dtype: float64

In [36]:
transact2.isnull().sum()

DAY_DT                   0
LOC_IDNT                 0
DBSKU                    0
FULL_PRICE_IND           0
TOTAL_SALES              0
TOTAL_UNITS              0
TOTAL_SALES_PRFT         0
TOTAL_COST               0
UNIT_SALES_PRICE         0
CITY                117416
STATE               117416
STORE_TYPE          117416
POSTAL_CD           117416
STORE_SIZE          117416
dtype: int64

In [38]:
transact2.shape

(12713513, 14)

In [43]:
transact2[transact2["CITY"].isnull()].head()

,DAY_DT,LOC_IDNT,DBSKU,FULL_PRICE_IND,TOTAL_SALES,TOTAL_UNITS,TOTAL_SALES_PRFT,TOTAL_COST,UNIT_SALES_PRICE,CITY,STATE,STORE_TYPE,POSTAL_CD,STORE_SIZE
584,2015-08-29,2865,2989343.0,0,13.89,1.0,-8.83,22.72,13.89,NaN,NaN,NaN,NaN,NaN
585,2015-09-03,2865,2100693.0,0,44.80,1.0,24.80,20.00,44.80,NaN,NaN,NaN,NaN,NaN
588,2015-08-20,2865,2105411.0,0,52.80,1.0,32.85,19.95,52.80,NaN,NaN,NaN,NaN,NaN
589,2015-09-10,2865,2111260.0,1,56.50,1.0,35.50,21.00,56.50,NaN,NaN,NaN,NaN,NaN
596,2015-08-19,2887,2984039.0,0,20.30,1.0,-1.70,22.00,20.30,NaN,NaN,NaN,NaN,NaN


In [44]:
#transact2[transact2["CITY"].isnull()].to_csv("missingstores.csv")

In [45]:
transact2[transact2["CITY"].isnull()].nunique()

DAY_DT              1075
LOC_IDNT              17
DBSKU               1883
FULL_PRICE_IND         2
TOTAL_SALES         6704
TOTAL_UNITS            8
TOTAL_SALES_PRFT    9723
TOTAL_COST          1877
UNIT_SALES_PRICE    6127
CITY                   0
STATE                  0
STORE_TYPE             0
POSTAL_CD              0
STORE_SIZE             0
dtype: int64

### After analysis of NA values we see that these occurred on stores with size =1 or NA/0

We will drop these values

In [48]:
transact2.shape

(12713513, 14)

In [49]:
transact2=transact2.dropna()

### Visualize new joined data set, start identifying anomalies if any

In [51]:
transact2.shape

(12596097, 14)

In [57]:
transact2.hist(bins=35)

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x000001C659922EF0>,
      dtype=object)

In [56]:
pp.ProfileReport(transact2).to_file(outputfile="./transact2.html")

#### This join looks acceptable on an initial analysis basis, we proceed to introduce product data to the dataset

## Merge Transaction and Product data

In [58]:
transact2=transact2.merge(product1,how='left',on="DBSKU")

In [59]:
transact2.shape

(12596097, 19)

In [60]:
round(100*(transact2.isnull().sum()/len(transact2.index)), 2)

DAY_DT              0.00
LOC_IDNT            0.00
DBSKU               0.00
FULL_PRICE_IND      0.00
TOTAL_SALES         0.00
TOTAL_UNITS         0.00
TOTAL_SALES_PRFT    0.00
TOTAL_COST          0.00
UNIT_SALES_PRICE    0.00
CITY                0.00
STATE               0.00
STORE_TYPE          0.00
POSTAL_CD           0.00
STORE_SIZE          0.00
DEPARTMENT          0.05
CLASS               0.05
SUBCLASS            0.05
SUBCLASS_NAME       0.05
DSUBCLASS           0.05
dtype: float64

In [61]:
transact2.isnull().sum()

DAY_DT                 0
LOC_IDNT               0
DBSKU                  0
FULL_PRICE_IND         0
TOTAL_SALES            0
TOTAL_UNITS            0
TOTAL_SALES_PRFT       0
TOTAL_COST             0
UNIT_SALES_PRICE       0
CITY                   0
STATE                  0
STORE_TYPE             0
POSTAL_CD              0
STORE_SIZE             0
DEPARTMENT          6466
CLASS               6466
SUBCLASS            6466
SUBCLASS_NAME       6466
DSUBCLASS           6466
dtype: int64

#### Analyze NA values

In [62]:
transact2[transact2["DEPARTMENT"].isnull()].nunique()

DAY_DT               825
LOC_IDNT             767
DBSKU                  4
FULL_PRICE_IND         1
TOTAL_SALES         1772
TOTAL_UNITS           17
TOTAL_SALES_PRFT    2269
TOTAL_COST           104
UNIT_SALES_PRICE    1607
CITY                 707
STATE                 48
STORE_TYPE            12
POSTAL_CD            754
STORE_SIZE           631
DEPARTMENT             0
CLASS                  0
SUBCLASS               0
SUBCLASS_NAME          0
DSUBCLASS              0
dtype: int64

In [68]:
transact2[transact2["DEPARTMENT"].isnull()].head()

,DAY_DT,LOC_IDNT,DBSKU,FULL_PRICE_IND,TOTAL_SALES,TOTAL_UNITS,TOTAL_SALES_PRFT,TOTAL_COST,UNIT_SALES_PRICE,CITY,STATE,STORE_TYPE,POSTAL_CD,STORE_SIZE,DEPARTMENT,CLASS,SUBCLASS,SUBCLASS_NAME,DSUBCLASS
3968,2015-10-10,1025,2339580.0,0,27.00,1.0,5.73,21.27,27.00,QUINCY,IL,Regional Mall,62301.0,3275.0,NaN,NaN,NaN,NaN,NaN
4999,2015-08-21,1220,848861.0,0,23.00,1.0,2.94,20.06,23.00,CASPER,WY,Power Strip,82609.0,2754.0,NaN,NaN,NaN,NaN,NaN
15232,2015-10-26,665,143271.0,0,19.20,1.0,-8.57,27.77,19.20,HAMDEN,CT,Strip Store,6514.0,3708.0,NaN,NaN,NaN,NaN,NaN
16348,2015-10-18,1061,2641068.0,0,24.99,1.0,4.79,20.20,24.99,ORLANDO,FL,Outlet Mall,32819.0,3170.0,NaN,NaN,NaN,NaN,NaN
19093,2015-08-03,9,143271.0,0,11.02,1.0,-16.75,27.77,11.02,AVON,MA,Strip Store,2322.0,3798.0,NaN,NaN,NaN,NaN,NaN


#### We see that these missing values arise due to SKUs which are part of a test class, and have been removed from the product data set, and can be removed safely

In [69]:
transact2=transact2.dropna()

In [70]:
transact2.shape

(12589631, 19)

In [71]:
transact2.head()

,DAY_DT,LOC_IDNT,DBSKU,FULL_PRICE_IND,TOTAL_SALES,TOTAL_UNITS,TOTAL_SALES_PRFT,TOTAL_COST,UNIT_SALES_PRICE,CITY,STATE,STORE_TYPE,POSTAL_CD,STORE_SIZE,DEPARTMENT,CLASS,SUBCLASS,SUBCLASS_NAME,DSUBCLASS
0,2015-09-26,1218,466896.0,0,16.80,1.0,1.3000,15.5000,16.80,COLUMBIA,SC,Lifestyle Center,29229.0,3050.0,10.0,3.0,31.0,2.0,1031.0
1,2015-08-02,1218,412445.0,0,29.99,1.0,12.9900,17.0000,29.99,COLUMBIA,SC,Lifestyle Center,29229.0,3050.0,10.0,2.0,21.0,2.0,1021.0
2,2015-10-21,1218,491738.0,1,44.00,1.0,28.2500,15.7500,44.00,COLUMBIA,SC,Lifestyle Center,29229.0,3050.0,10.0,5.0,50.0,3.0,1050.0
3,2015-08-02,1218,414979.0,0,24.00,1.0,6.9936,17.0064,24.00,COLUMBIA,SC,Lifestyle Center,29229.0,3050.0,10.0,3.0,32.0,1.0,1032.0
4,2015-07-26,1218,458372.0,1,48.00,1.0,30.0000,18.0000,48.00,COLUMBIA,SC,Lifestyle Center,29229.0,3050.0,10.0,3.0,32.0,1.0,1032.0


In [72]:
pp.ProfileReport(transact2).to_file(outputfile="./combined_dataset.html")

### Output to CSV

In [73]:
transact2.to_csv('tmc_final_data.csv',index=False)